In [19]:
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops

In [7]:
def downsample_image(image_path, scale_percent):
    image = cv2.imread(image_path)
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    new_dim = (width, height)
    resized_image = cv2.resize(image, new_dim, interpolation=cv2.INTER_LINEAR)
    
    return resized_image

# Criteria

## Color Complextiy

In [15]:
# How many types of colors are used
def count_unique_colors(image_path):
    image = cv2.imread(image_path)
    image = downsample_image(image_path, 0.75)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)  

    image = image // 40 * 40  
    unique_colors = np.unique(image.reshape(-1, image.shape[2]), axis=0)
    
    return len(unique_colors)

# Degree of color gradient and contrast
def calculate_color_entropy(image_path):
    image = cv2.imread(image_path)
    image = downsample_image(image_path, 0.75)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    
    hist = cv2.calcHist([image], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])
    hist_norm = hist / hist.sum()
    entropy = -np.sum(hist_norm[hist_norm>0] * np.log2(hist_norm[hist_norm>0]))
    
    return entropy

## Textures and details

In [26]:
# Types and complexity of textures in images
def texture_analysis(image_path):
    image = cv2.imread(image_path)
    image = downsample_image(image_path, 0.75)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    glcm = graycomatrix(image, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], 256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')
    homogeneity = graycoprops(glcm, 'homogeneity')
    
    return contrast, homogeneity

# Richness of image detail
def edge_density(image_path):
    image = cv2.imread(image_path)
    image = downsample_image(image_path, 0.75)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Sobel(image, cv2.CV_64F, 1, 1, ksize=5)
    edge_density = np.mean(edges)
    
    return edge_density


In [5]:
image_path = 'dataset/flower.JPG'
num_colors = count_unique_colors(image_path)
entropy = calculate_color_entropy(image_path)
contrast, homogeneity = texture_analysis(image_path)
edge_density = edge_density(image_path)

# Evaluation(distortion)

In [45]:
import os
import matplotlib.pyplot as plt


In [93]:
def cal_distortion(depth_dir, cut_dir, cut_file_name):

    depth = np.load(depth_dir+cut_file_name+'.npy')[0, :, :]
    layers_files = [r+'/'+f for r, d, fs in os.walk(cut_dir+cut_file_name) for f in fs if f.endswith('l.npy')]
    depth_step = np.max(depth)/len(layers_files)
    cut_depth = [i*depth_step for i in range(len(layers_files))]
    layer_masks = [np.load(f) for f in layers_files]

    distortion = 0
    for i, mask in enumerate(layer_masks):
        distortion += np.mean(depth[mask] - cut_depth[i])

    return distortion

In [94]:
depth_dir = './dataset/depth/'
depth_paths = [ f for r, d, fs in os.walk(depth_dir) for f in fs]
cut_dir = './output-sam/'
cut_file_names = [ d for r, ds, fs in os.walk(cut_dir) for d in ds]

In [97]:
for cut_name in cut_file_names:
    print(cut_name, cal_distortion(depth_dir, cut_dir, cut_name))

gardenCat 14.269725799560547
torri 3.8592071533203125
snowmountain2 -31.516761779785156
lake -30.327421188354492
snowmountain -7.62620735168457
summer 18.326998710632324
spring -11.579082489013672
